This lab on Logistic Regression is a Python adaptation of p. 161-163 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. Adapted by R. Jordan Crouser at Smith College for SDS293: Machine Learning (Spring 2016).

In [1]:
import pandas as pd
import numpy as np

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score

%matplotlib inline

# 4.6.3 Linear Discriminant Analysis
Let's return to the `Smarket` data from `ISLR`. 

In [19]:
df = pd.read_csv('Data/Smarket.csv', index_col=0, parse_dates=True)
df.head()

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,
2001-01-01,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2001-01-01,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2001-01-01,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
2001-01-01,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
2001-01-01,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


Now we will perform LDA on the `Smarket` data from the `ISLR` package. In `Python`, we can fit a LDA model using the `LinearDiscriminantAnalysis()` function, which is part of the `discriminant_analysis` module of the `sklearn` library. As we did with logistic regression and KNN, we'll fit the model using only the observations before 2005, and then test the model on the data from 2005.

In [21]:
X_train = df[:'2004'][['Lag1','Lag2']]
y_train = df[:'2004']['Direction']

X_test = df['2005':][['Lag1','Lag2']]
y_test = df['2005':]['Direction']

lda = LinearDiscriminantAnalysis()
model = lda.fit(X_train, y_train)

print(model.priors_)

[0.49198397 0.50801603]


The LDA output indicates prior probabilities of ${\hat{\pi}}_1 = 0.492$ and ${\hat{\pi}}_2 = 0.508$; in other words,
49.2% of the training observations correspond to days during which the
market went down.

In [22]:
print(model.means_)

[[ 0.04279022  0.03389409]
 [-0.03954635 -0.03132544]]


The above provides the group means; these are the average
of each predictor within each class, and are used by LDA as estimates
of $\mu_k$. These suggest that there is a tendency for the previous 2 days’
returns to be negative on days when the market increases, and a tendency
for the previous days’ returns to be positive on days when the market
declines. 

In [23]:
print(model.coef_)

[[-0.05544078 -0.0443452 ]]


The coefficients of linear discriminants output provides the linear
combination of `Lag1` and `Lag2` that are used to form the LDA decision rule.

If $−0.0554\times{\tt Lag1}−0.0443\times{\tt Lag2}$ is large, then the LDA classifier will
predict a market increase, and if it is small, then the LDA classifier will
predict a market decline. **Note**: these coefficients differ from those produced by `R`.
    
The `predict()` function returns a list of LDA’s predictions about the movement of the market on the test data:

In [24]:
pred=model.predict(X_test)
print(np.unique(pred, return_counts=True))

(array(['Down', 'Up'], dtype='<U4'), array([ 70, 182]))


The model assigned 70 observations to the "Down" class, and 182 observations to the "Up" class. 

Let's check out the confusion matrix to see how this model is doing. We'll want to compare the **predicted class** (which we can find in `pred`) to the **true class** (found in `y\_test})$.

#### sklearn.metrics.confusion_matrix
sklearn.metrics.confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)


#### sklearn.metrics.classification_report
sklearn.metrics.classification_report(y_true, y_pred, *, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False, zero_division='warn')

In [29]:
print(confusion_matrix(y_test, pred))
print()
print(classification_report(y_test, pred, digits=3))

[[ 35  76]
 [ 35 106]]

              precision    recall  f1-score   support

        Down      0.500     0.315     0.387       111
          Up      0.582     0.752     0.656       141

    accuracy                          0.560       252
   macro avg      0.541     0.534     0.522       252
weighted avg      0.546     0.560     0.538       252



# 4.6.4 Quadratic Discriminant Analysis
We will now fit a QDA model to the `Smarket` data. QDA is implemented
in `sklearn` using the `QuadraticDiscriminantAnalysis()` function, which is again part of the `discriminant_analysis` module. The
syntax is identical to that of `LinearDiscriminantAnalysis()`.

In [31]:
qda = QuadraticDiscriminantAnalysis()
model2 = qda.fit(X_train, y_train)
print(model2.priors_)
print()
print(model2.means_)

[0.49198397 0.50801603]

[[ 0.04279022  0.03389409]
 [-0.03954635 -0.03132544]]


The output contains the group means. But it does not contain the coefficients
of the linear discriminants, because the QDA classifier involves a
_quadratic_, rather than a linear, function of the predictors. The `predict()`
function works in exactly the same fashion as for LDA.

In [34]:
y_test.value_counts()

Up      141
Down    111
Name: Direction, dtype: int64

In [33]:
pred2=model2.predict(X_test)
print(np.unique(pred2, return_counts=True))
print()
print(confusion_matrix(y_test, pred2))
print()
print(classification_report(y_test, pred2, digits=3))

##                 Predicted 
##                 Down          Up
## Truth  Down      30           81
##        Up        20           121

## For Down: precision = (30)/(30+20) = 0.6
## For Down: recall = (30)/(30+81) = 0.270
## For Down: f1-score = 2*(precision * recall)/(precision + recall) = 2* (0.6*0.270)/(0.6+0.270)=0.3724138

## For Up: precision = (121)/(121+81) = 0.599
## For Up: recall = (121)/(20+121) = 0.858
## For Down: f1-score = 2*(precision * recall)/(precision + recall) = 2* (0.599*0.858)/(0.599+0.858)=0.7054798

## Accuracy = (30+121)/(252) =0.5992063

## Macro-average precision = (P1+P2)/2 = (0.600+0.599)/2 = 0.5995 ~= 0.600
## Macro-average recall = (R1+R2)/2 = (0.270+0.858)/2 = 0.564
## Macro-average f1-score = ((f1_1)+(f1_2))/2 = (0.373+0.706)/2 = 0.5395

# Up      111
# Down    141
# Total = 252

## Weighted-average precision = w1*P1 + w2*P2 = (111/252)*0.600 + (141/252)*0.599 = 0.5994405
## Weighted-average recall = w1*R1 + w2*R2 = (111/252)*0.270 + (141/252)*0.858 = 0.599
## Weighted-average f1-score = w1*(f1_1) + w2*(f1_2) = (111/252)*0.373 + (141/252)*0.706 = 0.5593214

(array(['Down', 'Up'], dtype=object), array([ 50, 202]))

[[ 30  81]
 [ 20 121]]

              precision    recall  f1-score   support

        Down      0.600     0.270     0.373       111
          Up      0.599     0.858     0.706       141

    accuracy                          0.599       252
   macro avg      0.600     0.564     0.539       252
weighted avg      0.599     0.599     0.559       252



Interestingly, the QDA predictions are accurate almost 60% of the time,
even though the 2005 data was not used to fit the model. This level of accuracy
is quite impressive for stock market data, which is known to be quite
hard to model accurately. 

This suggests that the quadratic form assumed
by QDA may capture the true relationship more accurately than the linear
forms assumed by LDA and logistic regression. However, we recommend
evaluating this method’s performance on a larger test set before betting
that this approach will consistently beat the market!

# An Application to Carseats Data
Let's see how the `LDA/QDA` approach performs on the `Carseats` data set, which is
included with `ISLR`. 

Recall: this is a simulated data set containing sales of child car seats at 400 different stores.

In [35]:
df2 = pd.read_csv('Data/Carseats.csv')
df2.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


See if you can build a model that predicts `ShelveLoc`, the shelf location (Bad, Good, or Medium) of the product at each store. Don't forget to hold out some of the data for testing!

In [ ]:
# Your code here


To get credit for this lab, please post your answers to the prompt in [#lab5](https://sds293.slack.com/messages/C7CR96LJ3).